# Solución etapa 4 - EDA + Cleaning

In [3]:
# Utilidades para print
from utils.print_utils import tabl, headr, titl, ptitl
from utils.explore_utils import explr

In [1]:
# Cargar los datasets
import pandas as pd

historicos_ordenes = pd.read_csv('../data/original/historicos_ordenes.csv')
registros_condiciones = pd.read_csv('../data/original/registros_condiciones.csv')
caracteristicas_equipos = pd.read_csv('../data/original/caracteristicas_equipos.csv')


## Exploración

In [4]:
ptitl('historicos_ordenes')
explr(historicos_ordenes)

ptitl('registros_condiciones')
explr(registros_condiciones)

ptitl('caracteristicas_equipos')
explr(caracteristicas_equipos)


** HISTORICOS_ORDENES **


Columnas:

['ID_Orden', 'ID_Equipo', 'Fecha', 'Tipo_Mantenimiento', 'Costo_Mantenimiento', 'Duracion_Horas', 'Ubicacion']

Numéricas

         ID_Orden    ID_Equipo    Costo_Mantenimiento    Duracion_Horas
-----  ----------  -----------  ---------------------  ----------------
count    10020       10020                    9971           10020
mean      4990.54      249.282                5255.82           23.9449
std       2892.6       143.828                4708.39           13.5833
min          1           1                     101.51            1
25%       2485.75      125                    2564.19           12
(8, 4)

Categóricas

        Fecha                Tipo_Mantenimiento    Ubicacion
------  -------------------  --------------------  -----------
count   10020                10020                 10020
unique  10000                2                     4
top     2020-01-01 16:00:00  Preventivo            Planta Sur
freq    2                    503

In [ ]:
# Profiling historicos_ordenes

from ydata_profiling import ProfileReport

profile = ProfileReport(historicos_ordenes, title="historicos_ordenes Profiling Report",
                        correlations={
                            "pearson": {"calculate": True},
                            "cramers": {"calculate": True},
                        })

In [ ]:
profile.to_notebook_iframe()

In [ ]:
# Profiling caracteristicas_equipos

print(titl('caracteristicas_equipos'))

profile = ProfileReport(caracteristicas_equipos, title="caracteristicas_equipos Profiling Report",
                        correlations={
                            "pearson": {"calculate": True},
                            "cramers": {"calculate": True},
                        })

In [ ]:
profile.to_notebook_iframe()

In [ ]:
# Profiling registros_condiciones

print(titl('registros_condiciones'))

profile = ProfileReport(registros_condiciones, title="registros_condiciones Profiling Report",
                        correlations={
                            "pearson": {"calculate": True},
                            "cramers": {"calculate": True},
                        })

In [ ]:
profile.to_notebook_iframe()

In [ ]:
ptitl('Dimensiones:')
print('historicos_ordenes:',historicos_ordenes.shape)
print('registros_condiciones:',registros_condiciones.shape)
print('caracteristicas_equipos:',caracteristicas_equipos.shape)

## Limpiar

In [ ]:
# hacemo copia
historicos_ordenes_cleaned = historicos_ordenes.copy()
registros_condiciones_cleaned = registros_condiciones.copy()
caracteristicas_equipos_cleaned = caracteristicas_equipos.copy()

### Nulos

In [ ]:
# Verificar valores nulos
print(titl("Valores nulos"))

print(headr('historicos_ordenes:'),historicos_ordenes_cleaned.isnull().sum())
print(headr('registros_condiciones:'),registros_condiciones_cleaned.isnull().sum())
print(headr('caracteristicas_equipos:'),caracteristicas_equipos_cleaned.isnull().sum())

In [ ]:
# Manejo de valores nulos
historicos_ordenes_cleaned.dropna(subset=['Costo_Mantenimiento'], inplace=True)  # Ejemplo de imputación
registros_condiciones_cleaned.dropna(subset=['Temperatura_C', 'Vibracion_mm_s'], inplace=True)  # Eliminar registros con valores faltantes críticos

In [ ]:
print(titl("Valores nulos after:"))

print(headr('historicos_ordenes:'),historicos_ordenes_cleaned.isnull().sum())
print(headr('registros_condiciones:'),registros_condiciones_cleaned.isnull().sum())
print(headr('caracteristicas_equipos:'),caracteristicas_equipos_cleaned.isnull().sum())

### Duplicados

In [ ]:
# Eliminar duplicados
print(titl("Duplicados"))

print(headr('historicos_ordenes:'),historicos_ordenes_cleaned.duplicated().sum())
print(headr('registros_condiciones:'),registros_condiciones_cleaned.duplicated().sum())
print(headr('caracteristicas_equipos:'),caracteristicas_equipos_cleaned.duplicated().sum())


In [ ]:
# Eliminar duplicados
historicos_ordenes_cleaned.drop_duplicates(inplace=True)
registros_condiciones_cleaned.drop_duplicates(inplace=True)
caracteristicas_equipos_cleaned.drop_duplicates(inplace=True)


In [ ]:
# Eliminar duplicados
print(titl("Duplicados después"))

print(headr('historicos_ordenes:'),historicos_ordenes_cleaned.duplicated().sum())
print(headr('registros_condiciones:'),registros_condiciones_cleaned.duplicated().sum())
print(headr('caracteristicas_equipos:'),caracteristicas_equipos_cleaned.duplicated().sum())


### Outliers

In [ ]:
# Detectar y manejar outliers (ejemplo para temperatura)

# Extraer columnas numéricas y categóricas
from sklearn.compose import make_column_selector as selector

def extract_numerical_and_categorical(df):
    numerical_columns_selector = selector(dtype_exclude=object)
    categorical_columns_selector = selector(dtype_include=object)

    numerical_columns = numerical_columns_selector(df)
    categorical_columns = categorical_columns_selector(df)

    return numerical_columns, categorical_columns


In [ ]:
h_num, h_cat = extract_numerical_and_categorical(historicos_ordenes_cleaned)
print(headr("h_num, h_cat"),h_num, h_cat)
r_num, r_cat = extract_numerical_and_categorical(registros_condiciones_cleaned)
print(headr("r_num, r_cat"),r_num, r_cat)
c_num, c_cat = extract_numerical_and_categorical(caracteristicas_equipos_cleaned)
print(headr("c_num, c_cat"),c_num, c_cat)

In [ ]:
# identificación
def identif_outliers(df, num_cols):
    IQR = df[num_cols].quantile(0.75) - df[num_cols].quantile(0.25)
    lower_bound = df[num_cols].quantile(0.25) - (IQR * 3)
    upper_bound = df[num_cols].quantile(0.75) + (IQR * 3)

    outliers = df[num_cols][(df[num_cols] < lower_bound) | (df[num_cols] > upper_bound)]

    # columnas con outliers
    columns_with_outliers = outliers.columns[outliers.count() > 0]
    columns_with_outliers

    return outliers.count(), columns_with_outliers, lower_bound, upper_bound

In [ ]:
h_nout, h_col_out, h_low, h_upp = identif_outliers(historicos_ordenes_cleaned, h_num)
print(headr("h_num_out:"),h_nout)

r_nout, r_col_out, r_low, r_upp = identif_outliers(registros_condiciones_cleaned, r_num)
print(headr("r_num_out:"),r_nout)

c_nout, c_col_out, c_low, c_upp = identif_outliers(caracteristicas_equipos_cleaned, c_num)
print(headr("c_num_out:"),c_nout)

In [ ]:
# Función para winsorizar una columna
def winsorize_column(column, lower_bound, upper_bound):
    return column.clip(lower=lower_bound, upper=upper_bound)

In [ ]:
# procesamos todas las comumnas con outliers
for col_name in h_col_out:
    historicos_ordenes_cleaned[col_name] = winsorize_column(historicos_ordenes_cleaned[col_name], h_low[col_name], h_upp[col_name])

for col_name in r_col_out:
    registros_condiciones_cleaned[col_name] = winsorize_column(registros_condiciones_cleaned[col_name], r_low[col_name], r_upp[col_name])

for col_name in c_col_out:
    caracteristicas_equipos_cleaned[col_name] = winsorize_column(caracteristicas_equipos_cleaned[col_name], c_low[col_name], c_upp[col_name])


In [ ]:
h_nout, h_col_out, h_low, h_upp = identif_outliers(historicos_ordenes_cleaned, h_num)
print(headr("h_num_out:"),h_nout)

r_nout, r_col_out, r_low, r_upp = identif_outliers(registros_condiciones_cleaned, r_num)
print(headr("r_num_out:"),r_nout)

c_nout, c_col_out, c_low, c_upp = identif_outliers(caracteristicas_equipos_cleaned, c_num)
print(headr("c_num_out:"),c_nout)

## Guardar estado

In [ ]:
historicos_ordenes_cleaned.to_csv('../data/cleaned/historicos_ordenes_cleaned.csv', index=False)
registros_condiciones_cleaned.to_csv('../data/cleaned/registros_condiciones_cleaned.csv', index=False)
caracteristicas_equipos_cleaned.to_csv('../data/cleaned/caracteristicas_equipos_cleaned.csv', index=False)